In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models, datasets
from torch.utils.data import DataLoader
import os
import random



In [3]:
# Define data transforms for training and testing
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}

data_dir = 'C:\\Users\\safar.NB-SAFAR\\Desktop\\Doc\\s\\project\\Dataset_Znacky\\Train-adj'  # Update with your dataset path

# Create a full dataset
full_dataset = datasets.ImageFolder(data_dir, data_transforms['train'])

# Create custom datasets for training and testing with 800 training images and 200 testing images per class
train_dataset = []
test_dataset = []

num_training_images = 800
num_testing_images = 200

for class_index in range(len(full_dataset.classes)):
    class_images = full_dataset.targets
    class_indexes = [i for i, c in enumerate(class_images) if c == class_index]

    # Randomly shuffle the class indexes
    random.shuffle(class_indexes)

    train_indexes = class_indexes[:num_training_images]
    test_indexes = class_indexes[num_training_images:num_training_images + num_testing_images]

    train_dataset.extend([full_dataset[i] for i in train_indexes])
    test_dataset.extend([full_dataset[i] for i in test_indexes])

In [4]:
# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Load the pre-trained ResNet model
model = models.resnet18(pretrained=True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(os.listdir(data_dir)))

# Set the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


c:\Users\safar.NB-SAFAR\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\safar.NB-SAFAR\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
# Training loop
num_epochs = 10
device = torch.device("cpu")#("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}")


Epoch 1/10, Loss: 0.8996604029337565
Epoch 2/10, Loss: 0.27368534394850336
Epoch 3/10, Loss: 0.1707882664569964
Epoch 4/10, Loss: 0.11742268587617824
Epoch 5/10, Loss: 0.0844480190674464
Epoch 6/10, Loss: 0.0626324775752922
Epoch 7/10, Loss: 0.04918213923306515
Epoch 8/10, Loss: 0.0375934910599608
Epoch 9/10, Loss: 0.02874637011360998
Epoch 10/10, Loss: 0.02219635585982663


In [6]:
# Test the model on the testing data
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 95.33%
